# Test download scripts


In [1]:
%reset -f

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import os
import sys
import random
import re
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from typing import Optional
from bs4 import BeautifulSoup
from time import sleep
from parfive import Downloader
from parfive import SessionConfig


In [4]:
# Set filepaths
PROJ = Path(os.path.realpath("."))
if str(PROJ) == "/n/home10/shreyasgm":
    PROJ = Path(
        "/n/holystore01/LABS/hausmann_lab/lab/glocal_aggregations/shreyas/proj/2023-02-05 - Pipeline"
    )
ROOT = PROJ.parents[1]
DATA = ROOT / "data/"


In [6]:
# Import custom modules
sys.path.append(str(PROJ))
sys.path.append(str(PROJ / "download_raster/viirs"))
sys.path.append(str(ROOT / "src/"))
from general_utils import *
from download_viirs import *


In [7]:
# List available rasters
rasters_root_url = "https://eogdata.mines.edu/nighttime_light/annual/v21/"
years_list = np.arange(2012, 2022)
rasters_year_urls = {x: rasters_root_url + str(x) + "/" for x in years_list}


In [8]:
a = []


In [9]:
if a:
    print("Check")


In [14]:
# Set an environment variable to store your password
os.environ["EOG_PASSWORD"] = "your_password"

In [18]:
# Get download links
download_links = list_available_rasters(endswith_filter=".median_masked.dat.tif.gz")


In [38]:
# Get files to download
download_dir = DATA / "raw/rasters/viirs/vnl_v2.1"
files_to_download = get_files_to_download(download_links, download_dir)
# Get headers
headers = get_headers()


In [39]:
len(files_to_download)

1

In [40]:
files_to_download[0:2]

['https://eogdata.mines.edu/nighttime_light/annual/v21/2017/VNL_v21_npp_2017_global_vcmslcfg_c202205302300.median_masked.dat.tif.gz']

In [41]:
# Use parfive to download files
dl = Downloader(max_conn=1, max_splits=3, progress=True, overwrite=False)
for file_to_queue in files_to_download:
    dl.enqueue_file(file_to_queue, path=download_dir, headers=headers)
# Download
res = dl.download()


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

VNL_v21_npp_2017_global_vcmslcfg_c202205302300.median_masked.dat.tif.gz:   0%|          | 0.00/287M [00:00<?, …

In [42]:
res.errors

[]

In [43]:
# if any files error out, retry
if res.errors:
    sleep(5)
    dl.retry(res)

In [46]:
# Unpack each file
for file in tqdm(list(download_dir.glob("*.gz"))):
    # Check if already unzipped
    if not file.with_suffix("").exists():
        unpack_file(file)


100%|██████████| 11/11 [05:50<00:00, 31.87s/it]
